In [19]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
from scipy.sparse import vstack, csr_matrix, save_npz, load_npz
import warnings
warnings.filterwarnings("ignore")
import gc

In [2]:
dict_dtypes = {
    'IsBeta': np.int8,                                                              
    'RtpStateBitfield': np.int8,                                                    
    'IsSxsPassiveMode': np.int8,                                                    
    'DefaultBrowsersIdentifier': np.int16,                                          
    'AVProductStatesIdentifier': np.int32,                                          
    'AVProductsInstalled': np.int8,                                                 
    'AVProductsEnabled': np.int8,
    'HasTpm': np.int8,                                                              
    'CountryIdentifier': np.int16,
    'CityIdentifier': np.int32,                                                     
    'OrganizationIdentifier': np.int8,
    'GeoNameIdentifier': np.int16,                                                  
    'LocaleEnglishNameIdentifier': np.int16,                                        
    'OsBuild': np.int16,                                                            
    'OsSuite': np.int16,                                                            
    'IsProtected': np.int8,                                                         
    'AutoSampleOptIn': np.int8,                                                     
    'SMode': np.int8,                                                               
    'IeVerIdentifier': np.int16,                                                    
    'Firewall': np.int8,                                                            
    'UacLuaenable': np.int32,                                                       
    'Census_OEMNameIdentifier': np.int16,                                           
    'Census_OEMModelIdentifier': np.int32,                                          
    'Census_ProcessorCoreCount': np.int16,
    'Census_ProcessorManufacturerIdentifier': np.int8,                              
    'Census_ProcessorModelIdentifier': np.int16,
    'Census_PrimaryDiskTotalCapacity': np.int32,                                    
    'Census_SystemVolumeTotalCapacity': np.int32,                                   
    'Census_HasOpticalDiskDrive': np.int8,                                          
    'Census_TotalPhysicalRAM': np.int32,                                            
    'Census_InternalPrimaryDiagonalDisplaySizeInInches': np.float16,                  
    'Census_InternalPrimaryDisplayResolutionHorizontal': np.int16,                  
    'Census_InternalPrimaryDisplayResolutionVertical': np.int16,
    'Census_InternalBatteryNumberOfCharges': np.int32,
    'Census_OSBuildNumber': np.int16,                                               
    'Census_OSBuildRevision': np.int32,                                             
    'Census_OSInstallLanguageIdentifier': np.int8,                                  
    'Census_OSUILocaleIdentifier': np.int16,                                        
    'Census_IsPortableOperatingSystem': np.int8,                                    
    'Census_IsFlightingInternal': np.int8,                                          
    'Census_IsFlightsDisabled': np.int8,                                            
    'Census_ThresholdOptIn': np.int8,                                               
    'Census_FirmwareManufacturerIdentifier': np.int16,
    'Census_FirmwareVersionIdentifier': np.int32,                                   
    'Census_IsSecureBootEnabled': np.int8,                                          
    'Census_IsWIMBootEnabled': np.int8,                                             
    'Census_IsVirtualDevice': np.int8,
    'Census_IsTouchEnabled': np.int8,                                               
    'Census_IsPenCapable': np.int8,                                                 
    'Census_IsAlwaysOnAlwaysConnectedCapable': np.int8,                             
    'Wdft_IsGamer': np.int8,
    'Wdft_RegionIdentifier': np.int8,
    'HasDetections': np.int8,
    'Census_InternalBatteryType_informed': np.int8,                                 
    'ProductName_index': np.int8,                                                   
    'Platform_index': np.int8,
    'Processor_index': np.int8,
    'OsPlatformSubRelease_index': np.int8,                                          
    'SkuEdition_index': np.int8,
    'PuaMode_index': np.int8,
    'SmartScreen_index': np.int8,
    'Census_MDC2FormFactor_index': np.int8,                                         
    'Census_DeviceFamily_index': np.int8,                                           
    'Census_ProcessorClass_index': np.int8,                                         
    'Census_PrimaryDiskTypeName_index': np.int8,
    'Census_ChassisTypeName_index': np.int8,
    'Census_PowerPlatformRoleName_index': np.int8,
    'Census_InternalBatteryType_index': np.int8,                                    
    'Census_OSArchitecture_index': np.int8,                                         
    'Census_OSBranch_index': np.int8,                                               
    'Census_OSEdition_index': np.int8,                                              
    'Census_OSSkuName_index': np.int8,                                              
    'Census_OSInstallTypeName_index': np.int8,                                      
    'Census_OSWUAutoUpdateOptionsName_index': np.int8,
    'Census_GenuineStateName_index': np.int8,                                       
    'Census_ActivationChannel_index': np.int8,
    'Census_FlightRing_index': np.int8,
    'OSBuild_diff': np.int8,
    'AvSigVersion_diff': np.int16,                                                  
    'OSBuild_fulldiff': np.int8,                                                    
    'AvSigVersion_fulldiff': np.int16,                                              
    'OsBuildLab_difftotal': np.int16,                                               
    'DateAvSigVersion_difftotal': np.int16,                                         
    'DateAvSigVersion_fulldifftotal': np.int16,
    'OsBuildLab_fulldifftotal': np.int16,
    'DateAvSigVersion_ratio': np.float16,                                              
    'OsBuildLab_ratio': np.float16,                                                    
    'DateAvSigVersion_fullratio': np.float16,
    'OsBuildLab_fullratio': np.float16,                                                
    'OsBuildLab_dayOfWeek': np.int8,                                                
    'AvSigVersion_dayOfWeek': np.int8, 
}

In [3]:
path = '../data/train_final_1'

In [4]:
allFiles = glob.glob(path + "/*.csv")

list_ = []
n = 0

for file_ in allFiles:
#     mask = file_['Platform_index'] != 0
    df = pd.read_csv(file_, dtype=dict_dtypes, low_memory=True)
    list_.append(df)
    if n%10==0:
        print(n)
    n+=1

train = pd.concat(list_, axis = 0, ignore_index = True)

0
10


In [5]:
train.shape

(8921483, 92)

In [48]:
train_sub = train.sample(4000000)

In [49]:
train_sub.shape

(4000000, 92)

In [50]:
sel_cols = [c for c in train.columns if c not in ['MachineIdentifier', 'HasDetections']]

In [51]:
X_train = train_sub.loc[:, sel_cols]
y_train = train_sub.loc[:,'HasDetections']

In [52]:
train_ids = X_train.index

In [53]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf.get_n_splits(train_ids, y_train)

5

In [54]:
lgb_model = lgb.LGBMClassifier(max_depth=-1,
                                   n_estimators=10000,
                                   learning_rate=0.05,
                                   num_leaves=2**12-1,
                                   colsample_bytree=0.28,
                                   objective='binary', 
                                   n_jobs=-1)

In [ ]:
counter = 0
for train_index, test_index in skf.split(train_ids, y_train):
    
    print('Fold {}\n'.format(counter + 1))
    
#     X_fit = vstack([train[train_index[i*m:(i+1)*m]] for i in range(train_index.shape[0] // m + 1)])
#     X_val = vstack([train[test_index[i*m:(i+1)*m]]  for i in range(test_index.shape[0] //  m + 1)])
#     X_fit, X_val = csr_matrix(X_fit, dtype='float32'), csr_matrix(X_val, dtype='float32')
    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
    
                              
    #xgb_model = xgb.XGBClassifier(max_depth=6,
    #                              n_estimators=30000,
    #                              colsample_bytree=0.2,
    #                              learning_rate=0.1,
    #                              objective='binary:logistic', 
    #                              n_jobs=-1)
    
                               
    lgb_model.fit(X_fit, y_fit, eval_metric='auc', 
                  eval_set=[(X_val, y_val)], 
                  verbose=100, early_stopping_rounds=20)
                  
    #xgb_model.fit(X_fit, y_fit, eval_metric='auc', 
    #              eval_set=[(X_val, y_val)], 
    #              verbose=1000, early_stopping_rounds=300)

    #lgb_train_result[test_index] += lgb_model.predict_proba(X_val)[:,1]
    #xgb_train_result[test_index] += xgb_model.predict_proba(X_val)[:,1]
    
    del X_fit, X_val, y_fit, y_val, train_index, test_index
    gc.collect()
    
    counter += 1

Fold 1

Training until validation scores don't improve for 20 rounds.
[100]	valid_0's auc: 0.729563	valid_0's binary_logloss: 0.607539
